<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/unigram_inverted_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNIGRAM INVERTED INDEX





**1. Building Unigram Inverted Index**

---





In [24]:
# importing files from drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Function to build an unigram inverted index

Structure of inverted index:
*   word1: {counts: x, documents: [doc1, dox2, doc3...]}
*   word2: {counts: y, documents: [doc3, doc4, doc5...]}
*   ...


    

In [25]:
import os

def unigram_inverted_index_builder(dataset_directory):
  unigram_inverted_index = {}
  list_file = os.listdir(dataset_directory)

  # Iterating through each file present in the directory
  for filename in list_file:
    if filename.endswith(".txt"):
      # Reading the content of the file
      file_path = os.path.join(dataset_directory, filename)
      with open(file_path, 'r') as file:
        content = file.read()

      # tokenizing the documnet to get unique tokens
      content_list = content.split()
      unique_content = set(content_list)

      # Updating the unigram inverted index
      for token in unique_content:
        # if the token is already in the inverted_index
        if token in unigram_inverted_index:
          unigram_inverted_index[token]['count'] += 1
          if filename not in unigram_inverted_index[token]['documents']:
            unigram_inverted_index[token]['documents'].append(filename)
        else:
          unigram_inverted_index[token] = {'count' : 1, 'documents': [filename]}
  return unigram_inverted_index


Invoking the unigram_inverted_index_builder function to build the inverted index

In [26]:
# location of preprocessed data set which is stored in the google drive
dataset_directory = "/content/drive/MyDrive/Information Retrieval/preprocessed_data"

# calling the builder function by passing the data set location
unigram_inverted_index = unigram_inverted_index_builder(dataset_directory)

# storing the newely generated unigram inverted index into a new text file
directory_name = "/content/drive/MyDrive/Information Retrieval"
text_file_name = "unigram_inverted_index_dataset.txt"
text_file = os.path.join(directory_name, text_file_name)

with open(text_file, 'w') as file:
  for term, info in unigram_inverted_index.items():
    file.write(f"{term}: {info}\n")

print("Unigram inverted index created")

Unigram inverted index created


Sorting the inverted index

In [27]:
# sorting the earlier generated unigram inverted index
sorted_items = sorted(unigram_inverted_index.items(), key = lambda x : x[0])
sorted_inverted_index = dict(sorted_items)

# storing the sorted inverted index into a new text file
sorted_text_file_name = "sorted_unigram_inverted_index_dataset.txt"
sorted_text_file = os.path.join(directory_name, sorted_text_file_name)

with open(sorted_text_file, 'w') as file:
  for term, info in sorted_inverted_index.items():
    file.write(f"{term} : {info}\n")

print("Sorted unigram inverted index created.")

Sorted unigram inverted index created.


# **2. Pickling the Unigram Inverted Index**

---



In [28]:
import pickle

pkl_file_name = "sorted_unigram_inverted_index.pkl"
pkl_file_path = os.path.join(directory_name, pkl_file_name)

with open(pkl_file_path, 'wb') as file:
  pickle.dump(sorted_inverted_index, file)

print("Sorted unigram inverted index pickled.")

Sorted unigram inverted index pickled.


# **3. Providing support for the query operations:**

---



1.   T1 **AND** T2
2.   T1 **OR** T2
3.   T1 **AND NOT** T2
4.   T1 **OR NOT** T2



Function to preprocess the query and return a preprocessed result

In [29]:
import string
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def preprocessing(query):
  # lower case the query
  query = query.lower()

  # tokenizing
  tokens = word_tokenize(query)

  # punctuation removal
  tokens = [word for word in tokens if word not in string.punctuation]

  # stopwords removal
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]

  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fuction to perform operation on query

In [43]:
import re

def sorted_posting_list(posting_list):
  # posting_list = ['file1.txt', 'file2.txt', 'file3.txt']
  # numeric part is needed

  pattern = re.compile(r'\d+')
  # storing the numeric part
  numeric_list = []

  for posting in posting_list:
    matches = pattern.search(posting)
    if matches:
      numeric_part = int(matches.group())
      numeric_list.append(numeric_part)
    else:
      numeric_list.append(0)

  numeric_list.sort()
  return numeric_list


def intersection_operation(posting_list_1, posting_list_2):
  list_1 = sorted_posting_list(posting_list_1)
  list_2 = sorted_posting_list(posting_list_2)

  # itersecting only the
  intersected_list = []

  pointer_1 = 0
  pointer_2 = 0
  while (pointer_1 < len(list_1) and pointer_2 < len(list_2)):
    if list_1[pointer_1] == list_2[pointer_2]:
      intersected_list.append(list_1[pointer_1])
      pointer_1 += 1
      pointer_2 += 1
    elif list_1[pointer_1] > list_2[pointer_2]:
      pointer_2 += 1
    else:
      pointer_1 += 1
  return intersected_list
# end of intersection function

# union function
def union_operation(posting_list_1, posting_list_2):
  list_1 = sorted_posting_list(posting_list_1)
  list_2 = sorted_posting_list(posting_list_2)

  # itersecting only the
  unioned_list = []

  pointer_1 = 0
  pointer_2 = 0

  while pointer_1 < len(list_1) and pointer_2 < len(list_2):
    if list_1[pointer_1] == list_2[pointer_2]:
      unioned_list.append(list_1[pointer_1])
      pointer_1 += 1
      pointer_2 +=1
    elif list_1[pointer_1] > list_2[pointer_2]:
      unioned_list.append(list_2[pointer_2])
      pointer_2 += 1
    else:
      unioned_list.append(list_1[pointer_1])
      pointer_1 += 1

  if pointer_1 < len(list_1):
    unioned_list.extend(list_1[pointer_1:])
  if pointer_2 < len(list_2):
    unioned_list.extend(list_2[pointer_2:])

  return unioned_list
# end of union function

Check Point: To check the working of operations on posting list

In [49]:
posting1 = ['file542.txt', 'file174.txt', 'file264.txt', 'file746.txt', 'file886.txt', 'file166.txt']
posting2 = ['file797.txt', 'file956.txt', 'file174.txt', 'file942.txt', 'file73.txt', 'file892.txt', 'file698.txt', 'file981.txt', 'file313.txt', 'file780.txt', 'file682.txt', 'file738.txt', 'file864.txt', 'file863.txt', 'file573.txt', 'file860.txt', 'file459.txt', 'file404.txt', 'file886.txt', 'file3.txt', 'file118.txt', 'file686.txt', 'file699.txt', 'file466.txt', 'file665.txt', 'file363.txt', 'file930.txt']

numeric_list1 = sorted_posting_list(posting1)
numeric_list2 = sorted_posting_list(posting2)
print("Sorted numeric posting list 1: ")
print(numeric_list1)
print("\nSorted numeric posting list 2: ")
print(numeric_list2)

intersect_list = intersection_operation(posting1, posting2)
print("\nIntersection :")
print(intersect_list)

print("\nUnion : ")
union_list = union_operation(posting1, posting2)
print(union_list)

Sorted numeric posting list 1: 
[166, 174, 264, 542, 746, 886]

Sorted numeric posting list 2: 
[3, 73, 118, 174, 313, 363, 404, 459, 466, 573, 665, 682, 686, 698, 699, 738, 780, 797, 860, 863, 864, 886, 892, 930, 942, 956, 981]

Intersection :
[174, 886]

Union : 
[3, 73, 118, 166, 174, 264, 313, 363, 404, 459, 466, 542, 573, 665, 682, 686, 698, 699, 738, 746, 780, 797, 860, 863, 864, 886, 892, 930, 942, 956, 981]


    ***Algorithm: (Not based on the length of the posting list)***
    #   implementing list as queue:
    #   query_tokens : [word1, word2, word3, word4]
    #   operator_list : [operator1, operator2, operator3]
    1.  creating a new list which will store the posting lists of the corresponding words
          query_postings = [posting_list_word_1, posting_list_word_2, posting_list_word_3, posting_list_word_4]
    2.  while operator_list is not empty:
    2.  perfomring the operation on query_postings[0] and query_postings[1] with operator_list[0] ALWAYS
          result_posting_list = posting_list_function(query_tokens[0], query_tokens[1], operators[0])
    3.  Pop out the query_postngs[0] and query_postings[1] as their work is done
    4.  pop out the operator_list[0] as well as its work is also done
    5.  Storing the resulted posting list at the 0th index of the query_postings[]

In [39]:
def operations_on_query(query_tokens, operator_list, inverted_index):
  # list to store the posting lists of corresponding query tokens (list of lists)
  posting_list = []
  for token in query_tokens:
    if token in inverted_index:
      posting_list.append(inverted_index[token]['documents'])
    else:
      posting_list.append([])

  # check point: printing token and their posting list
  for token, documents in zip(query_tokens, posting_list):
    print(token, ": ", documents)

  # performing operation on posting list and inverted index
  while(len(operator_list) > 0):

    posting_list_1 = posting_list.pop(0)
    posting_list_2 = posting_list.pop(0)

    operator = operator_list.pop(0)

    # removing the whitespaces
    operator = operator.strip()

    intermediate_posting_list = []

    # Operations on posting list
    if operator.lower() == 'and':
      intermediate_posting_list = intersection_operation(posting_list_1, posting_list_2)
    elif operator.lower() == "or":
      intermediate_posting_list = union_operation(posting_list_1, posting_list_2)
    elif operator.lower() == "and not":
      not_posting_list = difference_operation(posting_list_2)
      intermediate_posting_list = intersection_operation(posting_list_1, not_posting_list)
    elif operator.lower() == "or not":
      not_posting_list = difference_operation(posting_list_2)
      intermediate_posting_list = intersection_operation(posting_list_1, not_posting_list)

    # appending the intermediate posting list at the 0th index
    posting_list.insert(0, intermediate_posting_list)

  return posting_list[0]

In [41]:
# loading the pickled file
with open(pkl_file_path, 'rb') as file:
  inverted_index = pickle.load(file)

# input query
print("Enter input sequence: ")
query = input()

# input operations
print("Enter operations (AND, OR, AND NOT, OR NOT) seperated by comams:")
operations = input()
operators = operations.split(',')

# passing the query for preprocessing
query_tokens = preprocessing(query)

print("Preprocessed Tokens: " ,query_tokens)
print("Operations sequence: ", operators)

# Check point 1: Length of the token list is not 0
if len(query_tokens) == 0:
  print("Not a good query")

# Check point 2: If the there is only one token in preprocessed token list
elif len(query_tokens)  == 1:
  print("Query : ", query_tokens[0])

# Check point 3: operator list should be 1 less than token list for the operations
elif len(query_tokens) -1 != len(operators):
  print("Not a good operations sequence")

# Check point 4: other wise perform the operations
else:
  retrieved_document = operations_on_query(query_tokens, operators, inverted_index)
  print("Retrieved documents: ", retrieved_document)

Enter input sequence: 
car bag
Enter operations (AND, OR, AND NOT, OR NOT) seperated by comams:
and
Preprocessed Tokens:  ['car', 'bag']
Operations sequence:  ['and']
car :  ['file542.txt', 'file174.txt', 'file264.txt', 'file746.txt', 'file886.txt', 'file166.txt']
bag :  ['file797.txt', 'file956.txt', 'file174.txt', 'file942.txt', 'file73.txt', 'file892.txt', 'file698.txt', 'file981.txt', 'file313.txt', 'file780.txt', 'file682.txt', 'file738.txt', 'file864.txt', 'file863.txt', 'file573.txt', 'file860.txt', 'file459.txt', 'file404.txt', 'file886.txt', 'file3.txt', 'file118.txt', 'file686.txt', 'file699.txt', 'file466.txt', 'file665.txt', 'file363.txt', 'file930.txt']
Operator:  and
Posting list 1:  ['file542.txt', 'file174.txt', 'file264.txt', 'file746.txt', 'file886.txt', 'file166.txt']
Posting list 2:  ['file797.txt', 'file956.txt', 'file174.txt', 'file942.txt', 'file73.txt', 'file892.txt', 'file698.txt', 'file981.txt', 'file313.txt', 'file780.txt', 'file682.txt', 'file738.txt', 'file